In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from featout_exp.utils import CustomDataset, load_model, load_data

In [9]:
a = nn.CrossEntropyLoss(reduction='none') 

label = torch.Tensor([1,0,1,0]).long()
pred = torch.Tensor([[0,4], [0,1], [0,1], [0,1] ]).float()

a(pred, label)

tensor([0.0181, 1.3133, 0.3133, 1.3133])

In [2]:
data = load_data("train", 32, True)

for i in data:
    print(i)
    break

[tensor([[[[-0.4588, -0.4196, -0.3882,  ..., -0.0588, -0.1922, -0.2000],
          [-0.3490, -0.3647, -0.3490,  ..., -0.0824, -0.0431, -0.0824],
          [-0.3255, -0.3490, -0.3255,  ..., -0.1608, -0.0902, -0.0667],
          ...,
          [-0.1529, -0.1137, -0.0353,  ..., -0.5922, -0.6941, -0.7412],
          [-0.0824, -0.1059, -0.0745,  ..., -0.6314, -0.6157, -0.7176],
          [-0.1137, -0.0902, -0.0902,  ..., -0.5922, -0.5922, -0.6941]],

         [[-0.5059, -0.4667, -0.4353,  ..., -0.2078, -0.3333, -0.3020],
          [-0.4196, -0.4353, -0.4118,  ..., -0.2235, -0.1922, -0.2235],
          [-0.4039, -0.4353, -0.4039,  ..., -0.3020, -0.2471, -0.2471],
          ...,
          [-0.1686, -0.1294, -0.0510,  ..., -0.5922, -0.7176, -0.7490],
          [-0.1059, -0.1294, -0.0980,  ..., -0.6471, -0.6314, -0.6941],
          [-0.1608, -0.1216, -0.1216,  ..., -0.6314, -0.5922, -0.6784]],

         [[-0.4902, -0.4510, -0.4196,  ..., -0.2471, -0.3333, -0.2784],
          [-0.3961, -0.4118, 

In [7]:
# Reading the dataframe
df = pd.read_csv('csvs/second_half.csv', index_col= 0).iloc[:100]
(df.Prediction == df.Label).mean()

In [10]:
dataset = CustomDataset(dataframe=df)

# Create DataLoader to handle batching
batch_size = 32  # You can modify this as needed
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle= False)

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = load_model(device)

In [12]:
model.eval()  # Set the model to evaluation mode

all_predictions = []
all_true_labels = []
loss_per_image = []
criterion = nn.CrossEntropyLoss(reduction='none')  # Using 'none' to compute loss for each image

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        
        # Calculate loss for each image in the batch
        losses = criterion(outputs, labels)
        loss_per_image.extend(losses.cpu().numpy())
        
        # Get predicted labels
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())


In [14]:
test_df = df

In [15]:
test_df["loss"] = loss_per_image
test_df["pred_true"] =all_predictions